In [35]:
import cv2
import os
import csv
from glob import glob
from os.path import join
import mediapipe as mp
import numpy as np


In [28]:
dataset_path = join(os.curdir, "subsample")
processed_dataset_path = join(os.curdir, "subsample_processed")
if not os.path.isdir(processed_dataset_path):
    os.mkdir(processed_dataset_path)

class_labels = [
    "call", "dislike", "fist", "four", "like",
    "mute", "ok", "one", "palm", "peace",
    "peace_inverted", "rock", "stop", "stop_inverted",
    "three", "three2", "two_up", "two_up_inverted"
]

In [29]:
def get_data_list(path):
    '''
    Takes the path of a gesture and returns a 2D list containing the finger landmarks for all samples
    '''
    # Instantiate mediapipe hands object
    mp_hands = mp.solutions.hands
    # Select hand detection setting
    with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=1,
            min_detection_confidence=0.5) as hands:
        # Define a list to store all the hand location data
        data_list = []
        # Iterate over each image
        for idx, file in enumerate(path):
            # Read image
            image = cv2.imread(file)
            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            # Skip if no hand detected
            if not results.multi_hand_landmarks:
                continue
            # Check if Left hand is detected
            if results.multi_handedness[0].classification[0].label == 'Left':
                # Flip the image
                image = cv2.flip(image, 1)
                # Convert the BGR image to RGB before processing
                results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Skip if no hand detected
            if not results.multi_hand_landmarks:
                continue
            
            # Print handedness and draw hand landmarks on the image.
            # print('Handedness:', results.multi_handedness)
            
            landmark_list = []
            
            for i in range(21):
                landmark_list.append(results.multi_hand_landmarks[0].landmark[i].x)
                landmark_list.append(results.multi_hand_landmarks[0].landmark[i].y)
                landmark_list.append(results.multi_hand_landmarks[0].landmark[i].z)
            
            data_list.append(landmark_list)

    return data_list


In [33]:
for i, gesture in enumerate(class_labels):
    image_path = glob(join(dataset_path, gesture, "*jpg"))
    data_list = get_data_list(image_path)

    with open(join(processed_dataset_path, f"{gesture}.csv"), "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data_list)

    print(f"{gesture}: {len(data_list)} samples")


call: 89 samples
dislike: 81 samples
fist: 87 samples
four: 92 samples
like: 77 samples
mute: 82 samples
ok: 89 samples
one: 85 samples
palm: 91 samples
peace: 87 samples
peace_inverted: 98 samples
rock: 84 samples
stop: 88 samples
stop_inverted: 81 samples
three: 90 samples
three2: 94 samples
two_up: 86 samples
two_up_inverted: 81 samples


In [111]:
# Define dataset X and label y with a dummy row
X = np.zeros((1, 63))
y = np.zeros(1)
# Read and concatenate data
for i, gesture in enumerate(class_labels):
    data = np.loadtxt(join(processed_dataset_path, f"{gesture}.csv"),
                      delimiter=",", 
                      dtype=float)
    label = np.full(data.shape[0], i)
    # Concatenate into dataset and label
    X = np.concatenate((X, data), axis=0)
    y = np.concatenate((y, label))

# Remove the dummy row from dataset X
X = X[1:, :]
y = y[1:]
# Show dataset and label shape
print("X shape:", X.shape)
print("y shape:", y.shape)
# Export to csv
np.savetxt(join(processed_dataset_path, "X.csv"), X, delimiter=",")
np.savetxt(join(processed_dataset_path, "y.csv"), y, delimiter=",")


X shape: (1562, 63)
y shape: (1562,)
